In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store

In [4]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-04 19:20:27,675 INFO: Initializing external client
2025-03-04 19:20:27,683 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 19:20:29,489 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214705
Fetching data from 2025-02-04 00:20:27.672278+00:00 to 2025-03-04 23:20:27.672278+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.79s) 


In [5]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-04 01:00:00+00:00,2,0
1,2025-02-04 02:00:00+00:00,2,0
2,2025-02-04 03:00:00+00:00,2,0
3,2025-02-04 04:00:00+00:00,2,0
4,2025-02-04 05:00:00+00:00,2,0
...,...,...,...
174189,2025-03-04 18:00:00+00:00,263,132
174190,2025-03-04 19:00:00+00:00,263,99
174191,2025-03-04 20:00:00+00:00,263,86
174192,2025-03-04 21:00:00+00:00,263,76


In [6]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174194 entries, 0 to 174695
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         174194 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  174194 non-null  int32                  
 2   rides               174194 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 4.0 MB


In [7]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [8]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174194 entries, 0 to 174695
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         174194 non-null  datetime64[us]
 1   pickup_location_id  174194 non-null  int32         
 2   rides               174194 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 4.0 MB


In [9]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [10]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,2,0,0,0,0,3,0,0,3,5,...,0,3,0,1,0,1,0,0,255,2025-02-16 03:00:00
1,0,0,1,0,0,0,6,2,0,0,...,0,1,0,0,1,0,1,1,112,2025-02-13 12:00:00
2,2,1,0,0,1,1,0,1,0,1,...,0,0,0,1,0,0,2,0,218,2025-02-08 04:00:00
3,369,273,249,21,251,348,17,373,286,251,...,1,222,32,324,393,166,35,89,237,2025-02-08 19:00:00
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,57,2025-02-08 22:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,0,0,0,0,0,0,1,0,...,1,0,0,1,0,0,1,0,108,2025-02-23 15:00:00
247,0,0,0,2,0,0,4,0,1,0,...,0,3,2,0,0,0,2,0,71,2025-02-13 18:00:00
248,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,8,2025-02-22 07:00:00
249,0,1,1,1,1,1,2,0,4,0,...,1,2,4,1,0,0,0,0,168,2025-03-04 12:00:00


In [ ]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-04 19:20:47,534 INFO: Closing external client and cleaning up certificates.


In [ ]:
current_date

In [ ]:
features

In [ ]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

In [ ]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [ ]:
predictions

In [ ]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values